# Генерация текста при помощи LSTM

На этом семинаре поговорим про RNN. Здесь мы обучим модель на тексте Анны Карениной, а затем будем генерировать новый текст. **Эта модель сможет генерировать новый текст на основе нашего текста!**

Есть интересная [статья об RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), показывающая впечатляющие примеры того, что рекуррентные сети умеют делать. Ниже представлена ​​общая архитектура RNN.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="500">

Сначала загрузим ресурсы, необходимые для загрузки данных и создания модели.

In [ ]:
import datetime
from pathlib import Path
from tqdm.notebook import tqdm

import requests

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [ ]:
%load_ext tensorboard

In [ ]:
def get_datetime():
    return datetime.datetime.now().isoformat(sep='_', timespec='milliseconds').replace(':', '-')

## Загружаем данные

Затем мы загрузим текстовый файл Анны Карениной.

In [ ]:
text_path = Path('anna.txt')

if not text_path.exists():
    !wget https://github.com/hse-ds/iad-deep-learning/blob/66fb012/sem10/anna.txt -O $text_path

In [ ]:
# open text file and read in data as `text`
with text_path.open() as fp:
    text = fp.read()

In [ ]:
len(text)

Давайте проверим первые 100 символов, убедимся, что все красиво. 

In [ ]:
text[:100]

### Преобразуем символы в числа

В ячейках ниже постараемся создать пару **словарей** для преобразования символов в целые числа и обратно. Кодирование символов как целых чисел упрощает их использование в качестве входных данных в сети.

In [ ]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = sorted(set(text))
print(f'Number of distinct chars in text: {len(chars)}')
print(repr(''.join(chars)))

int2char = {i: ch for i, ch in enumerate(chars)}
char2int = {ch: i for i, ch in int2char.items()}

# encode the text
text_encoded = np.array([char2int[ch] for ch in text])

И мы можем видеть те же самые символы сверху, закодированные как целые числа.

In [ ]:
text_encoded[:100]

**Вопрос:** почему нельзя было просто воспользоваться функцией `ord()`?

In [ ]:
print([ord(c) for c in chars])

## One-hot encoding

Как вы можете видеть на изображении char-RNN выше, наш LSTM ожидает ввода, который  **one-hot encoded** , что означает, что каждый символ преобразуется в целое число (через наш созданный словарь), а затем преобразуется в столбец вектор, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте создадим для этого функцию!


In [ ]:
def one_hot_encode(arr, n_labels):
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [ ]:
# check that the function works as expected
test_seq = np.array([
    [3, 5, 1],
    [7, 2, 5],
])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

## Делаем батч-генератор


Для обучения на этих данных нужно создать мини-батчи для обучения. На простом примере наши батчи будут выглядеть так:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>


<br>

В этом примере возьмем закодированные символы (переданные как параметр `arr`) и разделим их на несколько последовательностей, заданных параметром `batch_size`. Каждая из наших последовательностей будет иметь длину `seq_length`.

### Собираем батчи

Наш батч-генератор будет работать следующим образом. Сначала мы выделим в тексте позиции, с которых будут начинаться подпоследовательности, которые станут элементами батчей. Такие подпоследовательности будут иметь одинаковую длину:

```
Everything was in confusion in the Oblonskys' house. The wife had discovered that the husband was carrying on an intrigue
↑               ↑               ↑               ↑               ↑               ↑               ↑               ↑               
```

Мы будем воспринимать текст как набор из таких подпоследовательностей:

```
Everything was i
n confusion in t
he Oblonskys' ho
use. The wife ha
d discovered tha
t the husband wa
s carrying on an
 intrigue
```

Последнюю подпоследовательность, которая будет слишком короткой, мы просто выбросим:

```
Everything was i
n confusion in t
he Oblonskys' ho
use. The wife ha
d discovered tha
t the husband wa
s carrying on an
```

Оставшиеся подпоследовательности мы перемешаем в случайном порядке:

```
use. The wife ha
t the husband wa
he Oblonskys' ho
d discovered tha
n confusion in t
Everything was i
s carrying on an
```

Наконец, мы разобьём их на батчи:

```
use. The wife ha
t the husband wa
he Oblonskys' ho

d discovered tha
n confusion in t
Everything was i

s carrying on an
```

и для простоты выбросим слишком короткие батчи:

```
use. The wife ha
t the husband wa
he Oblonskys' ho

d discovered tha
n confusion in t
Everything was i
```

Это то, что будет подаваться в модель на вход, то есть `x`. В нашем случае `y` будет теми же последовательностями, но сдвинутыми на один символ:

```
use. The wife ha         se. The wife had
t the husband wa    →     the husband was
he Oblonskys' ho         e Oblonskys' hou

d discovered tha          discovered that
n confusion in t    →     confusion in th
Everything was i         verything was in
```

In [ ]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    assert len(arr.shape) == 1

    # Сделайте массив start_indices из позиций начал подпоследовательностей в тексте
    <YOUR CODE>
    assert start_indices[-1] + seq_length + 1 <= len(arr)

    # Перемешайте его
    <YOUR CODE>

    # Сделайте так, чтобы количество элементов в нём делилось на batch_size
    <YOUR CODE>

    # Пройдите по start_indices скользящим окном с шириной и шагом batch_size
    for i in <YOUR CODE>:
        batch_start_indices = <YOUR CODE>

        # Сформируйте x и y
        <YOUR CODE>

        assert x.shape == (batch_size, seq_length), x.shape
        assert y.shape == (batch_size, seq_length), y.shape
        
        yield x, y

### Тестируем батч-генератор

Теперь создадим несколько наборов данных, и проверим, что происходит, когда мы батчуем данные.

In [ ]:
batches = get_batches(text_encoded, 8, 50)
x, y = next(batches)

In [ ]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])
assert (x[:, 1:] == y[:, :-1]).all(), 'y does not seem to be a shifted copy of x!'

Если вы правильно реализовали get_batches, результат должен выглядеть примерно так:
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 хотя точные цифры могут отличаться. Убедитесь, что данные сдвинуты на один шаг для `y`!!!

---
## Создаём модель


<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=500px>

Затем используем PyTorch для определения архитектуры сети. Начнем с определения слоев и операций, методов прямого прохода. 

### Устройство модели

В шаблоне `__init__()` уже прописано создание и сохранение необходимых словарей. Далее там предлагается описать следующую архитектуру:

* Слой LSTM, который принимает в качестве параметров:
    * `input_size`: размерность входа. В нашем случае это количество символов
    * `hidden_size`: размер скрытого слоя `hidden_size`
    * `num_layers`: количество слоёв ` num_layers`
    * `dropout`: вероятность дропаута `dropout_prob`
    * флаг `batch_first=True` (потому что модель на вход будет получать батчи с шейпом `batch x time x units`, а не `time x batch x units`)
* Слой Dropout с параметром `dropout_prob`
* Полносвязный слой с параметрами: размер ввода `hidden_size`, размер выхода — количество символов

Обратите внимание, что некоторые параметры были названы и указаны в функции `__init__`, их нужно сохранить и использовать, выполняя что-то вроде` self.dropout_prob = dropout_prob`.

In [ ]:
!nvidia-smi

In [ ]:
# check if GPU is available
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

if device == 'cpu':
    print('No GPU available, training on CPU; consider making n_epochs very small.')
else:
    print('Training on GPU!')

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, tokens, hidden_size=256, num_layers=2, dropout_prob=0.5):
        super().__init__()
        self.dropout_prob = dropout_prob
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = {i: ch for i, ch in enumerate(self.chars)}
        self.char2int = {ch: i for i, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        
        <YOUR CODE>
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm

        <YOUR CODE>
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes num_layers x batch_size x hidden_size,
        # initialized to zero, for hidden state and cell state of LSTM

        hidden_shape = (self.num_layers, batch_size, self.hidden_size)
        h0 = torch.zeros(hidden_shape, device=device)
        c0 = torch.zeros(hidden_shape, device=device)
        
        return h0, c0

In [ ]:
## TODO: set your model hyperparameters
# define and print the net
hidden_size = <YOUR CODE>
num_layers = <YOUR CODE>

net = CharRNN(chars, hidden_size=hidden_size, num_layers=num_layers)
print(net)

## Обучающий цикл

Во время обучения нужно установить количество эпох, скорость обучения и другие параметры.

Используем оптимизатор Адама и кросс-энтропию, считаем loss и, как обычно, выполняем back propagation!

Пара подробностей об обучении:

* На каждой итерации мы делаем `.detach()` на скрытом состоянии LSTM, чтобы не делать бэкпроп через всю историю обучения.
* Мы используем [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html), чтобы избавиться от взрывающегося градиента.

In [ ]:
def cross_entropy_batch_time_class(input, target):
    return F.cross_entropy(input.transpose(1, 2), target)

In [ ]:
def train(net, data, tb_dir, tb_tag=None, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, val_frequency=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        val_frequency: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    net = net.to(device)

    if tb_tag is None:
        tb_run_name = get_datetime()
    else:
        tb_run_name = f'{get_datetime()}_{tb_tag}'
    
    num_samples = 0
    with SummaryWriter(log_dir=str(tb_dir / tb_run_name)) as writer:
        for e in range(epochs):
            # initialize hidden state
            hidden = net.init_hidden(batch_size)
            
            with tqdm(get_batches(data, batch_size, seq_length), total=len(data) // (batch_size * seq_length)) as progress_bar:
                for x, y in progress_bar:
                    num_samples += batch_size

                    net.train()
                    
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, len(net.chars))
                    inputs = torch.tensor(x, device=device)
                    targets = torch.tensor(y, device=device, dtype=torch.int64)

                    # get the output from the model
                    output, hidden = net(inputs, hidden)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    hidden = tuple(h.detach() for h in hidden)
                    
                    # calculate the loss and perform backprop
                    loss = cross_entropy_batch_time_class(output, targets)

                    opt.zero_grad()  # zero accumulated gradients
                    loss.backward()
                    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
                    nn.utils.clip_grad_norm_(net.parameters(), clip)
                    opt.step()

                    writer.add_scalar('loss/train', loss.item(), num_samples)
                    
                    # loss stats
                    if (num_samples // batch_size) % val_frequency == 0:
                        with torch.no_grad():
                            net.eval()

                            # Get validation loss
                            val_hidden = net.init_hidden(batch_size)
                            val_losses = []
                            for x, y in get_batches(val_data, batch_size, seq_length):
                                # One-hot encode our data and make them Torch tensors
                                x = one_hot_encode(x, len(net.chars))
                                inputs = torch.tensor(x, device=device)
                                targets = torch.tensor(y, device=device, dtype=torch.int64)

                                output, val_hidden = net(inputs, val_hidden)
                                val_loss = cross_entropy_batch_time_class(output, targets)
                            
                                val_losses.append(val_loss.item())
                            
                            writer.add_scalar('loss/valid', np.mean(val_losses), num_samples)

## Создаём модель

### Задаём гиперпараметры

Теперь мы можем создать модель с заданными гиперпараметрами. Определим размеры мини-батчей!

In [ ]:
batch_size = <YOUR CODE>
seq_length = <YOUR CODE>
n_epochs = <YOUR CODE>  # start small if you are just testing initial behavior

In [ ]:
tb_dir = Path('tb_logs')

In [ ]:
%tensorboard --port 6006 --logdir $tb_dir

In [ ]:
# train the model
train(
    net, text_encoded, tb_dir,
    tb_tag=f'bs{batch_size}_sl{seq_length}_hs{hidden_size}_nl{num_layers}',
    epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, val_frequency=20)

## Улучшаем модель

Чтобы выбрать оптимальные гиперпараметры, нужно посмотреть на значения функции потерь на обучении и валидации. Если ваш лосс на обучении намного ниже, чем на тесте, то модель переобучена. Увеличьте регуляризацию или уменьшите число слоев в сети.

## Гиперпараметры

Гиперпараметры сети:

* `hidden_size` - Количество юнитов в скрытых слоях.
* `num_layers` - Количество используемых скрытых слоев LSTM.

В нашем примере вероятность отсева и скорость обучения сохраняется.

Для обучения:
* `batch_size` - количество объектов в батче, проходящих по сети за один проход.
* `seq_length` - Количество символов в последовательности, на которой обучается сеть. Обычно чем больше, тем лучше, сеть будет изучать более дальние зависимости.
* `lr` - learning rate.


 ## Советы и хитрости

> -  В глубоком обучении очень распространено запускать множество различных моделей с множеством различных настроек гиперпараметров и, в конце концов, использовать любую контрольную точку, дающую наилучшее качество на валидации.

> - Кстати, размер ваших тренировочных и проверочных разделов также является параметрами. Убедитесь, что у вас есть приличный объем данных в вашей валидационной выборки, иначе производительность проверки будет шумной и не очень информативной.

## Checkpoint

После обучения сохраним модель, чтобы можно было загрузить ее позже. Здесь сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

In [ ]:
# change the name, for saving multiple files
model_path = Path('lstm.pth')

checkpoint = {
    'hidden_size': net.hidden_size,
    'num_layers': net.num_layers,
    'state_dict': net.state_dict(),
    'tokens': net.chars,
}

with model_path.open('wb') as fp:
    torch.save(checkpoint, fp)

---
## Делаем предсказания

Теперь, когда модель обучена, сделаем предсказание следующих символов! Для предсказания мы передаем последний символ обучения, и сеть предсказывает следующий символ, который мы потом передаем обратно и получаем еще один предсказанный символ и так далее...

Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые наиболее вероятные символы $K$. Это не позволит сети выдавать нам совершенно абсурдные символы, а также позволит внести некоторый шум и случайность в выбранный текст. Узнать больше [можно здесь](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [ ]:
def predict(net, char, hidden, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x).to(device)
        
        # detach hidden state from history
        hidden = tuple(h.detach() for h in hidden)
        # get the output of the model
        out, hidden = net(inputs, hidden)

        # get the character probabilities
        char_probs = F.softmax(out, dim=-1).cpu().detach()

        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            # https://pytorch.org/docs/stable/generated/torch.topk.html
            char_probs, top_ch = char_probs.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        char_probs = char_probs.numpy().squeeze(axis=(0, 1))
        char = np.random.choice(top_ch, p=char_probs/char_probs.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], hidden

In [ ]:
predict(net, 'A', net.init_hidden(batch_size=1), top_k=5)[0]

### Инициализируем скрытое состояние и генерируем текст

Нужно создать скрытое состояние, чтобы сеть не генерировала произвольные символы. 

In [ ]:
def sample(net, size, prime='The', top_k=None):
    net = net.to(device)
    net.eval()
    
    # First off, run through the prime characters
    chars = list(prime)
    hidden = net.init_hidden(batch_size=1)
    for ch in prime:
        char, h = predict(net, ch, hidden, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for _ in range(size):
        char, hidden = predict(net, chars[-1], hidden, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(net, 1000, prime='Anna', top_k=5))

## Загружаем чекпойнт

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with Path('lstm.pth').open('rb') as fp:
    checkpoint = torch.load(fp)
    
loaded = CharRNN(checkpoint['tokens'], hidden_size=checkpoint['hidden_size'], num_layers=checkpoint['num_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

## Acknowledgements

Ноутбук основан на [ноутбуке для десятого семинара](https://github.com/hse-ds/iad-deep-learning/blob/66fb0128da4e65cb3260c088e2d462eb9d0c5eb1/sem10/sem10.ipynb) курса DL на майноре ИАД ВШЭ, который, в свою очередь, основан на [ноутбуке](https://github.com/udacity/deep-learning-v2-pytorch/blob/704a3e54b78afb8cd64d001d5160db93c986f63a/recurrent-neural-networks/char-rnn/Character_Level_RNN_Solution.ipynb) из курса Udacity Deep Learning.